In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
tf.set_random_seed(777)  # reproducibility

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [ ]:
X = tf.placeholder(tf.float32,shape=(None,784))
X_img = tf.reshape(X,(-1,28,28,1))
Y = tf.placeholder(tf.float32,shape=(None,10))

#hyperparameter
learning_rate = 0.01
training_epoches = 15
batch_size = 100


# layer1
W1 = tf.Variable(tf.random_normal([3,3,1,32],stddev=0.01))
L1 = tf.nn.conv2d(X_img,W1,strides=[1,1,1,1],padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME') # 14*14*32
L1 = tf.nn.dropout(L1,keep_prob=0.7)

# layer2
W2 = tf.Variable(tf.random_normal([3,3,32,64],stddev=0.01))
L2 = tf.nn.conv2d(L1,W2,strides=[1,1,1,1],padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME') # 7*7*64
L2 = tf.nn.dropout(L2,keep_prob=0.7)

# layer3
W3 = tf.Variable(tf.random_normal([3,3,64,128],stddev=0.01))
L3 = tf.nn.conv2d(L2,W3,strides=[1,1,1,1],padding='SAME')
L3 = tf.nn.relu(L3)
L3 = tf.nn.max_pool(L3,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME') # 4*4*128
L3 = tf.nn.dropout(L3,keep_prob=0.7)

# layer4
L3 = tf.reshape(L3,shape=[-1,4*4*128])
W4 = tf.get_variable('W4',shape=[4*4*128,625],initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([625]))
L4 = tf.matmul(L3,W4)+b4
L4 = tf.nn.dropout(L4,keep_prob=0.5)
# layer5
W5 = tf.get_variable('W5',shape=[625,10],initializer=tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([10]))
logits = tf.matmul(L4,W5) +b5

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

prediction = tf.argmax(logits,1)
is_correct= tf.equal(prediction,tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(is_correct,dtype=tf.float32))
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    total_batch = int(mnist.train.num_examples / batch_size)
    print('Start learning!')
    for epoch in range(training_epoches):
        avg_cost = 0.0
        for i in range(total_batch):
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)
            cost_val, _ = sess.run([cost,optimizer],feed_dict={
                X : batch_xs,
                Y : batch_ys
            })
            avg_cost += cost_val / total_batch
        print('Epoch : ',epoch+1,"cost",avg_cost)
    acc = sess.run(accuracy,feed_dict={
        X : mnist.test.images,
        Y : mnist.test.labels
    })
    print('Accuracy : ',acc)

Start learning!
Epoch :  1 cost 0.780198100914
Epoch :  2 cost 0.352388994612
Epoch :  3 cost 0.340854891762
